In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensor_forest.python import tensor_forest
from tensorflow.python.ops import resources
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [8]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=False)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [13]:
data = mnist.train.images
print data.shape
labels = mnist.train.labels
normalized_data = preprocessing.normalize(data)

(55000, 784)


In [4]:
test_data = mnist.test.images
normalized_test_data = preprocessing.normalize(test_data)
test_labels = mnist.test.labels

In [5]:
logisticRegr = LogisticRegression()
labels.shape

(55000,)

In [6]:
logisticRegr.fit(normalized_data, labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [7]:
predictions = logisticRegr.predict(normalized_test_data)

In [8]:
score = logisticRegr.score(normalized_test_data, test_labels)
print(score)

0.9154


# Random Forest

In [9]:
randomForestClf = RandomForestClassifier(n_estimators=30,n_jobs=2)

In [10]:
randomForestClf.fit(normalized_data, labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=2,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [11]:
rf_predictions = randomForestClf.predict(normalized_test_data)
rf_score = randomForestClf.score(normalized_test_data, test_labels)
print(rf_score)

0.9629


# Adverserial Examples for Random Forest

In [2]:
# Parameters
num_steps = 500 # Total steps to train
batch_size = 1024 # The number of samples per batch
num_classes = 10 # The 10 digits
num_features = 784 # Each image is 28x28 pixels
num_trees = 10
max_nodes = 1000

# Input and Target data
X = tf.placeholder(tf.float32, shape=[None, num_features])
# For random forest, labels must be integers (the class id)
Y = tf.placeholder(tf.float32, shape=[None])

# Random Forest Parameters
hparams = tensor_forest.ForestHParams(num_classes=num_classes,
                                      num_features=num_features,
                                      num_trees=num_trees,
                                      max_nodes=max_nodes).fill()

# Build the Random Forest
forest_graph = tensor_forest.RandomForestGraphs(hparams)
# Get training graph and loss
train_op = forest_graph.training_graph(X, Y)
loss_op = forest_graph.training_loss(X, Y)

INFO:tensorflow:Constructing forest with params = 
INFO:tensorflow:{'num_output_columns': 11, 'feature_bagging_fraction': 1.0, 'valid_leaf_threshold': 1, 'checkpoint_stats': False, 'initialize_average_splits': False, 'pruning_type': 0, 'prune_every_samples': 0, 'dominate_fraction': 0.99, 'max_fertile_nodes': 0, 'early_finish_check_every_samples': 0, 'dominate_method': 'bootstrap', 'bagging_fraction': 1.0, 'regression': False, 'param_file': None, 'bagged_num_features': 784, 'use_running_stats_method': False, 'max_nodes': 1000, 'split_finish_name': 'basic', 'leaf_model_type': 0, 'stats_model_type': 0, 'bagged_features': None, 'num_features': 784, 'split_after_samples': 250, 'num_outputs': 1, 'collate_examples': False, 'split_type': 0, 'num_classes': 10, 'num_splits_to_consider': 28.0, 'split_name': 'less_or_equal', 'finish_type': 0, 'inference_tree_paths': False, 'split_pruning_name': 'none', 'base_random_seed': 0, 'num_trees': 10, 'model_name': 'all_dense'}


In [10]:
infer_op, _, _ = forest_graph.inference_graph(X)
prediction = tf.nn.softmax(infer_op)
correct_prediction = tf.equal(tf.argmax(infer_op, 1), tf.cast(Y, tf.int64))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Minimize error using cross entropy
loss = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(prediction)))
# loss = tf.reduce_mean(tf.square( tf.cast(y_pred, tf.float64) - Y))
grad_input = tf.gradients(loss, X)[0]


In [23]:
# Initialize the variables (i.e. assign their default value) and forest resources
init_vars = tf.group(tf.global_variables_initializer(),
    resources.initialize_resources(resources.shared_resources()))

# Start TensorFlow session
sess = tf.Session()

# Run the initializer
sess.run(init_vars)

# Training
for i in range(1, num_steps + 1):
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    _, l = sess.run([train_op, loss_op], feed_dict={X: batch_x, Y: batch_y})
    if i % 50 == 0 or i == 1:
        acc = sess.run(accuracy_op, feed_dict={X: batch_x, Y: batch_y})
        print('Step %i, Loss: %f, Acc: %f' % (i, l, acc))

    infer_opp = sess.run([infer_op], feed_dict={X:mnist.test.images})
    infer_opp1 = np.asarray(infer_opp)[0]
    infer_opp = np.reshape(infer_opp1, (10000,10)) 
    y_test_pred = sess.run([prediction], feed_dict={infer_op: infer_opp}) 
    print (y_test_pred[0]) 
    grad = sess.run([grad_input], feed_dict={prediction:y_test_pred, Y:mnist.test.labels})    

Step 1, Loss: -1.000000, Acc: 0.416992
[[0.09679    0.10323935 0.09917413 ... 0.10297975 0.09831194 0.10494397]
 [0.10020212 0.09977435 0.103378   ... 0.09422161 0.10350913 0.09757658]
 [0.09450491 0.10899398 0.09976099 ... 0.10076496 0.09825937 0.10301112]
 ...
 [0.09534487 0.1013241  0.09619063 ... 0.09931202 0.1017089  0.10491975]
 [0.10148146 0.09535291 0.1015948  ... 0.09734996 0.10259605 0.10109536]
 [0.10820605 0.09411499 0.1034212  ... 0.09362537 0.1035713  0.09683612]]


ValueError: Cannot feed value of shape (1, 10000, 10) for Tensor u'Softmax_2:0', which has shape '(?, ?)'